In [1]:
import re
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, OneHotEncoder, MultiLabelBinarizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.feature_selection import SelectPercentile, f_regression
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

movies = pd.read_csv('movies_data.csv')

In [2]:
def convert_to_list(x):
    if isinstance(x, str) and x.strip():  # Check if x is a non-empty string
        return [item.strip() for item in x.split(',')]
    return ['not listed'] 

# Apply the conversion function to movies and production companies
movies['genres'] = movies['genres'].apply(convert_to_list)
movies['production_companies'] = movies['production_companies'].apply(convert_to_list)
movies.head()

,id,title,release_date,revenue,budget,production_companies,genres,popularity,vote_average,vote_count,overview,reviews,sentiment
0,5,Four Rooms,1995-12-09,4257354,4000000,"[Miramax, A Band Apart]",[Comedy],13.275,5.800,2618,It's Ted the Bellhop's first night on the job....,No reviews available,0.0000
1,6,Judgment Night,1993-10-15,12136938,21000000,"[Largo Entertainment, JVC, Universal Pictures]","[Action, Crime, Thriller]",9.284,6.500,331,"Four young friends, while taking a shortcut en...",No reviews available,0.0000
2,11,Star Wars,1977-05-25,775398007,11000000,"[Lucasfilm Ltd., 20th Century Fox]","[Adventure, Action, Science Fiction]",81.543,8.200,20457,Princess Leia is captured and held hostage by ...,"(As I'm writing this review, Darth Vader's the...",0.9998
3,12,Finding Nemo,2003-05-30,940335536,94000000,[Pixar],"[Animation, Family]",85.328,7.819,19102,"Nemo, an adventurous young clownfish, is unexp...",One of the best animated films I have ever see...,0.9976
4,13,Forrest Gump,1994-06-23,677387716,55000000,"[Paramount Pictures, The Steve Tisch Company, ...","[Comedy, Drama, Romance]",106.538,8.472,27251,A man with a low IQ has accomplished great thi...,Best movie ever. This is the best movie ever! ...,0.9963


In [3]:
movies['release_date'] = pd.to_datetime(movies['release_date'])

# Extract year, month, and week number as new columns
movies['release_year'] = movies['release_date'].dt.year
movies['release_month'] = movies['release_date'].dt.month
movies['release_week'] = movies['release_date'].dt.isocalendar().week  # Week number of the year

movies.head()

,id,title,release_date,revenue,budget,production_companies,genres,popularity,vote_average,vote_count,overview,reviews,sentiment,release_year,release_month,release_week
0,5,Four Rooms,1995-12-09,4257354,4000000,"[Miramax, A Band Apart]",[Comedy],13.275,5.800,2618,It's Ted the Bellhop's first night on the job....,No reviews available,0.0000,1995,12,49
1,6,Judgment Night,1993-10-15,12136938,21000000,"[Largo Entertainment, JVC, Universal Pictures]","[Action, Crime, Thriller]",9.284,6.500,331,"Four young friends, while taking a shortcut en...",No reviews available,0.0000,1993,10,41
2,11,Star Wars,1977-05-25,775398007,11000000,"[Lucasfilm Ltd., 20th Century Fox]","[Adventure, Action, Science Fiction]",81.543,8.200,20457,Princess Leia is captured and held hostage by ...,"(As I'm writing this review, Darth Vader's the...",0.9998,1977,5,21
3,12,Finding Nemo,2003-05-30,940335536,94000000,[Pixar],"[Animation, Family]",85.328,7.819,19102,"Nemo, an adventurous young clownfish, is unexp...",One of the best animated films I have ever see...,0.9976,2003,5,22
4,13,Forrest Gump,1994-06-23,677387716,55000000,"[Paramount Pictures, The Steve Tisch Company, ...","[Comedy, Drama, Romance]",106.538,8.472,27251,A man with a low IQ has accomplished great thi...,Best movie ever. This is the best movie ever! ...,0.9963,1994,6,25


In [77]:
# Calculate average revenue by production company
avg_revenue_by_company = movies.explode('production_companies').groupby('production_companies')['revenue'].mean()

# Sort companies by their average revenue
ranked_companies = avg_revenue_by_company.sort_values(ascending=False)

# Define thresholds for each category (you could use percentiles or fixed thresholds)
top_companies = ranked_companies.head(100)  # Top 100 companies by revenue
mid_companies = ranked_companies.iloc[100:1000]  # Mid-level companies
low_companies = ranked_companies.tail(3284-1100)  # Remaining companies

# Create a new column in your DataFrame to categorize each company
def categorize_company(company):
    if company in top_companies.index:
        return 'Top'
    elif company in mid_companies.index:
        return 'Mid-level'
    else:
        return 'Low-level'

# Function to categorize each movie based on the highest-ranking production company
def categorize_production_level(companies):
    categories = [categorize_company(company) for company in companies]
    # Define the ranking order for categories
    rank_priority = {'Top': 3, 'Mid-level': 2, 'Low-level': 1}
    # Get the highest rank based on priority
    highest_category = max(categories, key=lambda x: rank_priority[x])
    return highest_category

# Apply the categorization function to the list of production companies for each movie
movies['production_level'] = movies['production_companies'].apply(categorize_production_level)
movies = movies.drop(columns = ['production_companies', 'release_date'])
movies.head()

,id,title,revenue,budget,genres,popularity,vote_average,vote_count,overview,reviews,sentiment,release_year,release_month,release_week,production_level
0,5,Four Rooms,4257354,4000000,[Comedy],13.275,5.800,2618,It's Ted the Bellhop's first night on the job....,No reviews available,0.0000,1995,12,49,Mid-level
1,6,Judgment Night,12136938,21000000,"[Action, Crime, Thriller]",9.284,6.500,331,"Four young friends, while taking a shortcut en...",No reviews available,0.0000,1993,10,41,Mid-level
2,11,Star Wars,775398007,11000000,"[Adventure, Action, Science Fiction]",81.543,8.200,20457,Princess Leia is captured and held hostage by ...,"(As I'm writing this review, Darth Vader's the...",0.9998,1977,5,21,Top
3,12,Finding Nemo,940335536,94000000,"[Animation, Family]",85.328,7.819,19102,"Nemo, an adventurous young clownfish, is unexp...",One of the best animated films I have ever see...,0.9976,2003,5,22,Top
4,13,Forrest Gump,677387716,55000000,"[Comedy, Drama, Romance]",106.538,8.472,27251,A man with a low IQ has accomplished great thi...,Best movie ever. This is the best movie ever! ...,0.9963,1994,6,25,Top


In [18]:
# Step 1: Explode the 'production_companies' column and calculate average revenue and movie count
movies_exploded = movies.explode('production_companies')
company_stats = movies_exploded.groupby('production_companies').agg(
    avg_revenue=('revenue', 'mean'),
    movie_count=('production_companies', 'count')
).reset_index()

# Step 2: Apply log transformation to the revenue and movie count
company_stats['log_avg_revenue'] = np.log1p(company_stats['avg_revenue'])  # log(1 + revenue) to avoid log(0)
company_stats['log_movie_count'] = np.log1p(company_stats['movie_count'])  # log(1 + count) to handle 0s

# Step 3: Standardize the log-transformed features to balance their scale
scaler = StandardScaler()
company_stats[['log_avg_revenue', 'log_movie_count']] = scaler.fit_transform(company_stats[['log_avg_revenue', 'log_movie_count']])

# Step 4: Calculate the combined score based on standardized log-transformed revenue and movie count
# Weights are adjusted for their relative importance
alpha = 0.5  # Revenue weight
beta = 0.5  # Movie count weight

company_stats['score'] = alpha * company_stats['log_avg_revenue'] + beta * company_stats['log_movie_count']

# Step 5: Rank companies based on score and assign levels
top_threshold = company_stats['score'].quantile(0.9)  # Top 10% as top companies
mid_threshold = company_stats['score'].quantile(0.5)  # Middle 40% as mid-level companies

def classify_company(score):
    if score >= top_threshold:
        return 'Top-level'
    elif score >= mid_threshold:
        return 'Mid-level'
    else:
        return 'Low-level'

company_stats['production_level'] = company_stats['score'].apply(classify_company)

# Create a dictionary to map each company to its production level
production_level_map = company_stats.set_index('production_companies')['production_level'].to_dict()

# Step 6: Assign the highest production level to each movie's production companies
def categorize_production_level(companies):
    categories = [production_level_map.get(company, 'Low-level') for company in companies]
    rank_priority = {'Top-level': 3, 'Mid-level': 2, 'Low-level': 1}
    highest_category = max(categories, key=lambda x: rank_priority[x])
    return highest_category

movies['production_level'] = movies['production_companies'].apply(categorize_production_level)

# View the top 20 companies and their production levels
company_stats.head(20)

,production_companies,avg_revenue,movie_count,log_avg_revenue,log_movie_count,score,production_level
0,"""DIA"" Productions GmbH & Co. KG",4.435093e+07,1,0.451488,-0.513143,-0.030827,Mid-level
1,(Colossal) Pictures,5.332167e+07,1,0.535232,-0.513143,0.011045,Mid-level
2,11:11 Mediaworks,1.199751e+08,1,0.903898,-0.513143,0.195377,Mid-level
3,12 Gauge Productions,1.038000e+06,1,-1.255512,-0.513143,-0.884328,Low-level
4,120 Films,6.500000e+06,1,-0.421521,-0.513143,-0.467332,Low-level
5,1492 Pictures,4.261504e+08,10,1.480124,2.582494,2.031309,Top-level
6,16 Block Productions,6.566472e+07,1,0.629892,-0.513143,0.058374,Mid-level
7,1600 Limited Partnership,2.580471e+07,1,0.205280,-0.513143,-0.153931,Low-level
8,2 Entertain,3.731167e+07,1,0.372919,-0.513143,-0.070112,Low-level
9,20th Century Fox,1.415905e+08,183,0.979207,7.697932,4.338569,Top-level


### Fitting a Linear Regression Pipeline

In [78]:
# Step 1: Fit the MultiLabelBinarizer on the genres data (on the entire dataset)
mlb = MultiLabelBinarizer()
genres_encoded = mlb.fit_transform(movies['genres'])

# Create a DataFrame from the encoded genres
genres_df = pd.DataFrame(genres_encoded, columns=mlb.classes_, index=movies.index)

# Concatenate the genre DataFrame with the original dataset
movies = pd.concat([movies, genres_df], axis=1)

In [81]:
movie_titles = movies['title']
X = movies.drop(columns=['revenue', 'title', 'id', 'overview', 'reviews', 'genres'])  # Features
y = movies['revenue']  # Target

# Define the categorical and numerical columns
categorical_cols = ['production_level']  # Only 'production_level' as a categorical variable
numerical_cols = ['budget', 'popularity', 'vote_average', 'vote_count', 'sentiment','release_year', 'release_month', 'release_week'] + list(mlb.classes_)

# Define the preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),  # Scale numeric features
        ('cat', OneHotEncoder(), categorical_cols),  # One-hot encode categorical features
    ])

# Define the model pipeline
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the pipeline to the training data
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred)  

print(f'Mean Squared Error (Random Forest): {mse}')

Mean Squared Error (Random Forest): 5009883888307241.0
